In [33]:
import yfinance as yf
import pandas as pd
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm

tickers = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLV', 'XLY', 'XLU', 'XLRE', 'XLC']
prices = yf.download(tickers, start = '2015-07-22', end = '2020-07-22')['Close']

returns = prices.pct_change().dropna()

correlation_matrix = returns.corr()
correlation_matrix = correlation_matrix.rename_axis(None).rename_axis(None, axis = 1)
correlation_matrix = correlation_matrix.stack().reset_index()
correlation_matrix.columns = ['ETF', 'Pair', 'Correlation']

correlation_matrix =  correlation_matrix[correlation_matrix['ETF'] != correlation_matrix['Pair']]
correlation_matrix = correlation_matrix.sort_values(by = 'Correlation', ascending = False)
correlation_matrix = correlation_matrix.drop_duplicates('Correlation')

potential_pairs = correlation_matrix[correlation_matrix['Correlation'] > 0.8]

print("Potential Correlated Pairs:")
potential_pairs

/var/folders/qd/xclxh05s0x9509nblm6z434w0000gp/T/ipykernel_34555/3575719629.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  prices = yf.download(tickers, start = '2015-07-22', end = '2020-07-22')['Close']
[*********************100%***********************]  11 of 11 completed

Potential Correlated Pairs:


,ETF,Pair,Correlation
47,XLI,XLF,0.921375
4,XLB,XLI,0.921346
56,XLK,XLC,0.913406
65,XLK,XLY,0.913384
21,XLC,XLY,0.894735
33,XLF,XLB,0.893015
54,XLI,XLY,0.888986
110,XLY,XLB,0.866833
113,XLY,XLF,0.863268
104,XLV,XLK,0.855851


In [34]:
tickers = prices.columns
cointegrated_pairs = []
prices = prices.dropna()
candidate_pairs = []

for i in range(len(tickers)):
    for j in range(i + 1, len(tickers)):
        candidate_pairs.append([tickers[i], tickers[j]])

for i, j in candidate_pairs:
        etf1 = i
        etf2 = j

        score, pvalue, _ = coint(prices[etf1], prices[etf2])

        if pvalue < 0.1:
                cointegrated_pairs.append((etf1, etf2, pvalue))
                
cointegrated_pairs_df = pd.DataFrame(cointegrated_pairs, columns = ['ETF', 'Pair', 'P-Value'])
cointegrated_pairs_df = cointegrated_pairs_df.sort_values('P-Value').reset_index(drop=True)

print("Cointegration Test Results:")
cointegrated_pairs_df

Cointegration Test Results:


,ETF,Pair,P-Value
0,XLC,XLY,0.053237
1,XLF,XLI,0.072493


In [35]:
results = []

def zscore_calc(series):
    return (series - series.mean()) / series.std()

def adf_test(series):
    test_res = adfuller(series)
    return {'stat': test_res[0], 'p-value': test_res[1]}

def hedge_ratio_calc(series1, series2):
    x = sm.add_constant(series2)
    model = sm.OLS(series2, x).fit()

    return model.params.iloc[1]

for etf1, etf2 in candidate_pairs:
    series1 = prices[etf1]
    series2 = prices[etf2]

    hedge_ratio = hedge_ratio_calc(series1, series2)

    spread = series1 - (hedge_ratio * series2)

    zscore_spread = zscore_calc(spread)

    adf_res = adf_test(spread)

    results.append(
        {'ETF1': etf1,
         'ETF2': etf2,
         'adf_value': adf_res['stat'],
         'p-value': adf_res['p-value'],
         'mean': zscore_spread.mean(),
         'std': zscore_spread.std()}
    )

results = pd.DataFrame(results)
results = results.sort_values('p-value', ascending = True)
results = results[results['p-value'] < 0.1]

print("ADF Test Results:")
results

ADF Test Results:


,ETF1,ETF2,adf_value,p-value,mean,std
17,XLC,XLV,-3.129433,0.024450,8.375219e-16,1.0
27,XLF,XLI,-2.971806,0.037613,-1.715569e-15,1.0
14,XLC,XLP,-2.932012,0.041750,2.701683e-17,1.0
48,XLP,XLY,-2.825119,0.054774,-2.215380e-15,1.0
36,XLI,XLRE,-2.715493,0.071395,-5.403367e-17,1.0
5,XLB,XLP,-2.599160,0.093171,2.161347e-16,1.0
